In [ ]:
%load_ext aiida
%aiida

In [ ]:
# General imports.
from copy import deepcopy
import numpy as np
import re
import time
from datetime import datetime
import ipywidgets as ipw
from IPython.display import clear_output

from aiida.engine import ExitCode, ToContext, WorkChain, calcfunction,workfunction

In [ ]:
style = {'description_width': '160px'}
layout = {'width': '40%'}
output = ipw.Output()
output1 = ipw.Output()
output2 = ipw.Output()
add_person = ipw.Button(description='Add person')
people_html = ipw.HTML()
report_html = ipw.HTML()
cash_html = ipw.HTML()
info_html = ipw.HTML()

name = ipw.Text(description='Name or nickname',style=style, layout=layout)
email = ipw.Text(description='e-mail',style=style, layout=layout)
startdate = ipw.DatePicker(description='Starting from:',value=datetime.now(),style=style, layout=layout)

In [ ]:
html_phead = """
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:2px;font-family:Arial, sans-serif;font-size:14px;
    overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
    font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-dark{background-color:#c0c0c0;border-color:inherit;text-align:left;vertical-align:middle}
.tg .tg-llyw{background-color:#efefef;border-color:inherit;text-align:left;vertical-align:middle}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:middle}
</style>
<table class="tg">
<thead>
<tr>"""    
html_ptail =     "</tr></thead><tbody>"
html_report_head = html_phead + "<th class='tg-dark' >Date </th> <th class='tg-dark'>Description</th>" + html_ptail
#
html_people_head = html_phead + "<th class='tg-dark' >Who </th>"
html_people_head += "<th class='tg-dark' >Balance </th>"
html_people_head += "<th class='tg-dark' >Coffee </th>"
html_people_head += "<th class='tg-dark' >Other </th>"
html_people_head += "<th class='tg-dark' >Cash </th>"
html_people_head += "<th class='tg-dark' >Days </th>"
html_people_head += html_ptail
#
tclass = ["", "tg-dark", "tg-llyw"]   

In [ ]:
# validate e-mail

def validemail(email):
    pat = "^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+@[a-zA-Z0-9]+\.[a-z]{1,3}$"
    if re.match(pat,email):
        return True
    return False

In [ ]:
def timeconversion(intime):
    if isinstance(intime,float):
        return datetime.fromtimestamp(intime).strftime("%Y-%m-%d")
    else:
        return time.mktime(intime.timetuple())

In [ ]:
def daysactive(person):
    if 'left' in person.extras:
        return np.busday_count(timeconversion(person['started']),timeconversion(person.extras['left']))*person.extras['coeff']
    else:
        return np.busday_count(timeconversion(person['started']),datetime.now().strftime("%Y-%m-%d"))*person.extras['coeff']

In [ ]:
def check_people():
    global cash
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['nanotech@coffee_member']}})
    options = []
    emails = []
    for node in qb.all(flat=True):
        options.append((node['name'],node))
        emails.append(node['email'])
    who.options = options
    qb = QueryBuilder()
    qb.append(CalcFunctionNode, filters={
        'label': {'in': ['new_event']}})   
    ordered_entries =  sorted([node.outputs.result.get_dict() for node in qb.all(flat=True)], key=lambda d: d['datei'], reverse=True)
    personal = {}
    # sum of effective days for all members. Absences will be subtracted later
    total_days = 0
    for person in who.options:
        days = daysactive(person[1])
        total_days += days
        personal[person[0]] = {'coffee':0,'other':0, 'cash':0, 'effectivedays':days,'coeff':person[1].extras['coeff']}
    cash = 0
    html_report = html_report_head
    html_people = html_people_head
    total_cost=0
    odd = -1
    for entry in ordered_entries:
        what = entry['event']
        if 'cash' in entry['event']:
            cash += entry['amount']
            if 'Donated' not in what:
                personal[entry['person']]['cash'] += entry['amount']
        elif 'Bought' in entry['event']:
            total_cost += entry['amount']
            if 'coffee' in entry['event']:
                personal[entry['person']]['coffee'] += entry['amount']
            else:
                personal[entry['person']]['other'] += entry['amount']
        elif 'Absence' in entry['event']:
            tosubtract = entry['amount']*personal[entry['person']]['coeff']
            personal[entry['person']]['effectivedays'] -= tosubtract
            total_days -= tosubtract
        
        html_report += "<tr>"
        html_report += f"<td class={tclass[odd]}> {timeconversion(entry['datei'])} </td>"
        html_report += f"<td class={tclass[odd]}> {entry['description']} </td>"
        html_report += "</tr>" 
        odd *= -1   
     
    short_report = [(person,(personal[person]['coffee']+personal[person]['other']+personal[person]['cash'])-((total_cost - cash)*personal[person]['effectivedays'])/total_days,personal[person]['coffee'],personal[person]['other'],personal[person]['cash'],personal[person]['effectivedays']) for person in personal]
    short_report = sorted(short_report, key=lambda d: d[1], reverse=True)
    for person in short_report:
        html_people += "<tr>"
        html_people += f"<td class={tclass[odd]}> {person[0]} </td>"
        html_people += f"<td class={tclass[odd]}> {person[1]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[2]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[3]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[4]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[5]:.2f} </td>"
        html_people += "</tr>" 
        odd *= -1
    people_html.value = html_people
    report_html.value = html_report
    cash_html.value = f"<h3>Available cash: {cash:.2f} CHF</h3>"
    return emails,cash

In [ ]:
display(cash_html)

# Persons

In [ ]:
display(ipw.VBox([people_html,startdate,name,email, add_person,output,]))

In [ ]:
@calcfunction
def new_person(name,email,startdate):
    return Dict({'name':name.value,'email':email.value,'started':startdate.value})
    
def on_add_person_clicked(b):
    global emails,cash
    with output:
        clear_output()
        if validemail(email.value) and startdate.value is not None and name.value != '':
            if email.value in emails:
                print("person already present")
            else:
                new = new_person(Str(name.value),Str(email.value),Float(timeconversion(startdate.value)))
                new.label = 'nanotech@coffee_member'
                new.set_extra('coeff',1)
                print("Added",new['name'],'starting from: ',timeconversion(new['started']),'pk:',new.pk )
                emails.append(email.value)
    emails,cash = check_people()
            
add_person.on_click(on_add_person_clicked)

# Actions

In [ ]:
@workfunction
def create_action(action):
    action.label = 'nanotech@coffee_action'
    return action

@calcfunction
def new_event(person=None,datei=None,datef=None,event=None,description=None,amount=None,kgcoffee=None,available_cash=None):
        
    if 'Bought' in event.value or 'Added' in event.value:
        theamount = np.abs(amount.value)
        if 'coffee' in event.value:
            return Dict({'person': person['name'], 'datei':datei.value, 'amount':theamount, 'event':event.value,
                        'description': f"{person['name']} bought {kgcoffee.value}kg coffee ({description.value}) for {theamount} CHF" })
        else:    
            return Dict({'person': person['name'], 'datei':datei.value, 'amount':theamount, 'event':event.value,
                            'description': f"{person['name']} {event.value} ({description.value}) for {theamount} CHF" })
    elif event.value == 'Donated cash':
        theamount = np.abs(amount.value)
        return Dict({'person': person['name'], 'datei':datei.value, 'amount':theamount, 'event':event.value,
                        'description': f"{person['name']} donated {theamount} CHF ({description.value}) to the common fund" })
    elif event.value == 'Requested cash':
        theamount = np.abs(amount.value)
        if amount.value <= available_cash.value +0.01 :
            return Dict({'person': person['name'], 'datei':datei.value, 'amount': -1.0*theamount, 'event':event.value,
                            'description': f"{person['name']} received {theamount} CHF from the common fund" })
        else:
            return Dict({'person': person['name'], 'datei':datei.value,'amount':0.0,'event':event.value,
                            'description': f"{person['name']} requested {theamount} CHF from the common fund but they are not available. They will not be counted"})
    elif event.value == 'Absence':
        theamount = np.busday_count(timeconversion(datei.value),timeconversion(datef.value))
        return Dict({'person': person['name'], 'datei':datei.value,'amount':theamount,'event':event.value,
                        'description': f"{person['name']}  entered absence from {timeconversion(datei.value)}  until {timeconversion(datef.value)} ({theamount} days {description.value})" })
    

In [ ]:
event = ipw.Dropdown(options=[],
                     description='Possible actions',style=style, layout=layout)
amount = ipw.FloatText(description='CHF',style=style, layout=layout)
kgcoffee = ipw.FloatText(description='kg',style=style, layout=layout,value =1)
datei_widget = ipw.DatePicker(description='On:',value=datetime.now(),style=style, layout=layout)
datef_widget = ipw.DatePicker(description='Till:',value=datetime.now(),style=style, layout=layout)
description = ipw.Text(description='Description',style=style, layout=layout)
who = ipw.Dropdown(options=[],description='Person',style=style, layout=layout)
apply = ipw.Button(description='Apply')

def date_is_valid(date,person):
    if 'left' in person.extras:
        dateleft = person.extras['left']
    else:
        dateleft = float(3000000000)
    return person['started'] - 100000 < date < dateleft + 100000

def on_apply_clicked(b):
    global emails, cash
    with output2:
        clear_output()
        person = who.value
        datei = timeconversion(datei_widget.value)
        datef = datei
        if 'Absence' in event.value.value:
            datef = timeconversion(datef_widget.value)            
        kg = 0
        if 'coffee' in event.value.value:
            kg = kgcoffee.value
        if date_is_valid(datei,person) and date_is_valid(datef,person):
            what = new_event(person=person,datei=Float(datei),
                            datef=Float(datef),
                            event=event.value,description=Str(description.value),
                             amount=Float(amount.value),
                             kgcoffee=Float(kg),
                             available_cash=Float(cash)
                            )
            print("Added event: ",what['description']) 
        else:
            print("Date is not valid for this person") 
        emails,cash = check_people() 
apply.on_click(on_apply_clicked)

def on_event_change(change):
    with output1:
        clear_output()
        if 'Absence' in change['new'].value:
            todisplay = [who,datei_widget,datef_widget,description,info_html]
            description.value =''
            kgcoffee.value=0
            amount.value=0
            info_html.value=''
        elif 'Bought coffee' == change['new'].value:
            todisplay = [who,datei_widget,kgcoffee,description,amount,info_html]
            kgcoffee.value=2
            description.value='Konstanz: mondo verde'
            amount.value=60
            info_html.value=''
        else:
            description.value=''
            amount.value=0
            kgcoffee.value=0
            todisplay = [who,datei_widget,description,amount,info_html]
            info_html.value='To add/request/donate cash, send/request the amount via Twint at +41 76 53 25 330 <br> Donated cash does not change the balance of the donor, but it is added to the common fund'
            
        display(ipw.VBox(todisplay))
        
event.observe(on_event_change,names='value')

In [ ]:
qb = QueryBuilder()
qb.append(Node, filters={
    'label': {'in': ['nanotech@coffee_action']}})
needed_actions=['Bought coffee','Bought accessory','Bought cleaning stuff','Donated cash', 'Added cash','Requested cash','Absence']
existing_actions = {}    
for node in qb.all(flat=True):
    existing_actions[node.value]=node
    #print(node.value,node.pk)
for action in needed_actions:
    if action not in existing_actions:
        existing_actions[action]=create_action(Str(action))
event.options = list(existing_actions.items())


In [ ]:
emails,cash = check_people()
cash_html.value = f"<h3>Available cash: {cash:.2f} CHF</h3>"
display(ipw.VBox([event,output1,apply,output2]))

# Report

In [ ]:
display(report_html)

In [ ]:
if False :
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['nanotech@coffee_member']}})
    for node in qb.all(flat=True):
        print(node.pk,node.get_dict())

In [ ]:
if False :
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['nanotech@coffee_action']}})
    for node in qb.all(flat=True):
        print(node.pk, node.value)

In [ ]:
if False :
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['new_event']}})
    for node in qb.all(flat=True):
        print(node.pk, node.outputs.result['event'])

In [ ]:
#timeconversion(float(3000000000 - 100000))